In [73]:
import numpy as np
import pandas as pd
import re
from os import listdir
from os.path import isfile, join
import warnings
warnings.simplefilter(action = "ignore", category = Warning)

In [74]:
files = [f for f in listdir("raw_data/") if isfile(join("raw_data/", f)) and f.lower().endswith(".csv")]
files.sort()

In [87]:
def find_header_row(full_path):
    try:
        header_pattern = re.compile(r"^,[ \w-]+.*$")
        with open(full_path, "r") as f:
            for i, line in enumerate(f):
                if header_pattern.match(line.strip()):
                    return i
    except Exception as e:
        print("Failed to find header row in", full_path)
        print(e)

In [76]:
def clean_data(df):
    date_col = "Unnamed: 0" if "Unnamed: 0" in df.columns else " "
    df[date_col] = df[date_col].str.strip()
    df = df[df[date_col].str.len() > 4]
    df[date_col] = df[date_col].astype(str)
    df["Year"] = df[date_col].str[:4]
    df["Month"] = df[date_col].str[4:]
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors = "coerce")
    df = df.drop(columns = [date_col])
    df = df.dropna()
    df["Year"] = df["Year"].astype(int)
    df["Month"] = df["Month"].astype(int)
    return df

In [77]:
def get_clean_data(data_path):
    full_path = "raw_data/" + data_path
    header_row = find_header_row(full_path)
    try:
        data = pd.read_csv(full_path, skiprows = header_row, na_values = ["-99.99", "-999", -99.99, -999])
        df = clean_data(data)
        return df
    except Exception as e:
        print("Failed to read CSV for", data_path, "expected header row was", header_row)
        print(e)

In [78]:
def get_df_name_prefix(name):
    name = name.lower()
    if "industry_portfolios" in name:
        return "us_" + re.match(r"^(\d+)_", name).group(1)
    elif "25_portfolios" in name:
        if name == "25_portfolios":
            return "na_100"
        elif "developed_25_portfolios" in name:
            return "dev_100"
        elif "ex_us" in name:
            return "dev_exUS_100"
        elif "europe" in name:
            return "euro_100"
        elif "japan" in name:
            return "jap_100"
        elif "asia_pacific" in name:
            return "asia_100"
    elif "100_portfolios" in name:
        return "us_100"
    elif "data_factors" in name:
        return "us_ff3"
    elif "5_factors" in name:
        if "f-f" in name:
            return "us_ff5"
        elif "america" in name:
            return "na_ff5"
        elif "developed_5_factors" in name:
            return "dev_ff5"
        elif "ex_us" in name:
            return "dev_exUS_ff5"
        elif "europe" in name:
            return "euro_ff5"
        elif "japan" in name:
            return "jap_ff5"
        elif "asia_pacific" in name:
            return "asia_ff5"

In [79]:
def split_and_save(df, prefix, start_years, end_year = 2019):
    for start_year in start_years:
        sub_df = df[(df["Year"] >= start_year) & (df["Year"] <= end_year)]
        sub_df.to_csv(f"clean_data/{prefix}_{start_year}.csv")

In [80]:
def process_data(data_path):
    global curr_25_combo
    global curr_25_name
    
    combo_25_pattern = r"^(.*?25_Portfolios)"
    match_25 = re.search(combo_25_pattern, data_path)
    if match_25:
        curr_25_name = match_25.group(1)
        suffix_pattern = r"25_Portfolios(.*?)(?:\d+x\d+[_]*)?(?:\.csv|\.CSV)$"
        suffix = re.search(suffix_pattern, data_path).group(1)
        if suffix == "_":
            suffix = "5x5"
        if curr_25_combo is None:
            curr_25_combo = get_clean_data(data_path)
        else:
            df = get_clean_data(data_path)
            curr_25_combo = pd.merge(curr_25_combo, df, on = ["Year", "Month"], suffixes = (None, suffix))
    elif ("MOM" not in data_path) and ("Momentum" not in data_path):
        if curr_25_combo is not None:
            prefix = get_df_name_prefix(curr_25_name)
            assert prefix is not None, f"oopsie, {data_path}"
            split_and_save(curr_25_combo, prefix, [1991, 2008])
        curr_25_combo = None
        curr_25_name = None
        df = get_clean_data(data_path)
        prefix = get_df_name_prefix(data_path)
        assert prefix is not None, f"whoopsie, {data_path}"
        split_and_save(df, prefix, [1964, 1991, 2008])

In [81]:
curr_25_combo = None
curr_25_name = None

def process_loop():
    for file in files:
        process_data(file)

In [102]:
def combine_ff5_and_mom(clean_files):
    for file in files:
        name = file.lower()
        if "mom" in name:
            if "f-f" in name:
                tag = "us_ff5"
            elif "america" in name:
                tag = "na_ff5"
            elif "developed_mom_factor" in name:
                tag = "dev_ff5"
            elif "ex_us" in name:
                tag = "dev_exUS_ff5"
            elif "europe" in name:
                tag = "euro_ff5"
            elif "japan" in name:
                tag = "jap_ff5"
            elif "asia_pacific" in name:
                tag = "asia_ff5"
            momentum_df = get_clean_data(file)
            factors_data = [f for f in clean_files if tag in f]
            for f in factors_data:
                factors_df = pd.read_csv("clean_data/" + f)
                year = f.replace(".csv", "")[-4:]
                combined = pd.merge(factors_df, momentum_df, on = ["Year", "Month"], suffixes = (None, "mom"))
                combined.to_csv(f"clean_data/{tag}_mom_{year}.csv")

In [83]:
process_loop()

In [103]:
clean_files = [f for f in listdir("clean_data/") if isfile(join("clean_data/", f)) and f.lower().endswith(".csv")]
clean_files.sort()
combine_ff5_and_mom(clean_files)